In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("EDA").getOrCreate()

In [3]:
spark

In [6]:
# Loading data 
locations = spark.read.csv("../data/locations.csv", header=True, inferSchema=True)
transactions = spark.read.csv("../data/rental_transactions.csv", header=True, inferSchema=True) 
users = spark.read.csv("../data/users.csv", header=True, inferSchema=True)
vehicles = spark.read.csv("../data/vehicles.csv", header=True, inferSchema=True)

In [7]:
locations.show(5)

+-----------+--------------------+--------------------+---------+-----+--------+-----------+----------+
|location_id|       location_name|             address|     city|state|zip_code|   latitude| longitude|
+-----------+--------------------+--------------------+---------+-----+--------+-----------+----------+
|       2702|Jackson, Velazque...|3140 Heath Radial...|  Modesto|   CA|   94540|   86.25802| -169.2448|
|       4380|            Bean LLC|51144 Patrick Isl...|  Fontana|   CA|   92188|-74.4558925|-42.279882|
|       7709|     Gilbert-Simmons|    4738 Lewis Locks|Roseville|   CA|   91032|-65.4309305|-64.763489|
|       8607|    Coleman-Robinson|  324 Robin Causeway|  Modesto|   CA|   93714| -64.281076|-77.669631|
|       5499|        Deleon Group|    51725 Evans View|Roseville|   CA|   91849| 18.4951575|-154.76578|
+-----------+--------------------+--------------------+---------+-----+--------+-----------+----------+
only showing top 5 rows



In [8]:
transactions.show(5)

+----------+----------+----------+-------------------+-------------------+---------------+----------------+------------+
| rental_id|   user_id|vehicle_id|  rental_start_time|    rental_end_time|pickup_location|dropoff_location|total_amount|
+----------+----------+----------+-------------------+-------------------+---------------+----------------+------------+
|b139d8e1b2|320be8068b|0d52304987|2024-02-28 08:05:00|2024-03-01 05:05:00|           1497|            6785|       450.0|
|7afd60f6d3|320be8068b|975d72985c|2024-01-07 20:16:00|2024-01-09 21:16:00|           5345|            2608|      2450.0|
|733a9361bc|8f31b734a6|0d9f0f0fb9|2024-01-07 09:36:00|2024-01-07 17:36:00|           2546|            5442|        80.0|
|6e546b69dd|8f31b734a6|967fdab45e|2024-01-05 11:30:00|2024-01-07 04:30:00|           8147|            4380|      2050.0|
|acc192b64a|8f31b734a6|32d58ea4b7|2024-03-06 18:19:00|2024-03-09 14:19:00|           6290|            8932|      1360.0|
+----------+----------+---------

In [9]:
users.show(5)

+----------+----------+---------+--------------------+------------------+---------------------+---------------------+-------------+---------+
|   user_id|first_name|last_name|               email|      phone_number|driver_license_number|driver_license_expiry|creation_date|is_active|
+----------+----------+---------+--------------------+------------------+---------------------+---------------------+-------------+---------+
|26d08ab733|      Lisa|   Parker|lisa.parker@gmail...|334.271.2972x60554|             MO028963|           2033-06-21|   2024-05-26|        1|
|0a0430e6f9|  Courtney|   Martin|courtney.martin@y...|  826-262-0518x252|             VW966518|           2028-09-28|   2024-05-22|        0|
|eb5d10cccd|    Andrew|  Mcclain|andrew.mcclain@ho...|   +1-467-858-1702|             WL839491|           2028-09-01|   2024-01-29|        1|
|2a59127ee0|   Michael|   Hoover|michael.hoover@ya...|  001-220-342-6250|             UI603163|           2028-11-29|   2024-03-22|        1|
|e3a46

In [10]:
vehicles.show(5)

+------+----------------------+--------------------+----------------+---------------+---------------------+------------------+------------+---------------------+--------------------+----------+---------------------+---------+------------+
|active|vehicle_license_number|   registration_name|    license_type|expiration_date|permit_license_number|certification_date|vehicle_year|base_telephone_number|        base_address|vehicle_id|last_update_timestamp|    brand|vehicle_type|
+------+----------------------+--------------------+----------------+---------------+---------------------+------------------+------------+---------------------+--------------------+----------+---------------------+---------+------------+
|     1|               5818886|CITY,LIVERY,LEASI...|FOR HIRE VEHICLE|     27-09-2025|             6EPABCVK|        2018-01-09|        2018|        (646)780-0129|1515 THIRD STREET...|67789f742d|  04-06-2024 13:25:00|  Ferrari|    high_end|
|     1|               5520432|    FERNANDEZ

### Spark Job 1

In [12]:
#  Revenue per Location
revenue_per_location_df = transactions.groupBy("pickup_location").agg(F.sum("total_amount").alias("total_revenue"))
revenue_per_location_df.show(5)

+---------------+-------------+
|pickup_location|total_revenue|
+---------------+-------------+
|           1959|      58090.0|
|           8928|      43470.0|
|           1507|      57120.0|
|           8932|      53890.0|
|           9182|      63900.0|
+---------------+-------------+
only showing top 5 rows



In [14]:
# Total Transactions per Location
transactions_per_location_df = transactions.groupBy("pickup_location").agg(F.count("rental_id").alias("total_transactions"))
transactions_per_location_df.show(5)

+---------------+------------------+
|pickup_location|total_transactions|
+---------------+------------------+
|           1959|                69|
|           8928|                66|
|           1507|                77|
|           8932|                65|
|           9182|                75|
+---------------+------------------+
only showing top 5 rows



In [16]:
# Average, Max, and Min Transaction Amounts
transaction_amounts_df = transactions.groupBy("pickup_location").agg(
    F.avg("total_amount").alias("avg_transaction"),
    F.max("total_amount").alias("max_transaction"),
    F.min("total_amount").alias("min_transaction")
)
transaction_amounts_df.show(5)

+---------------+-----------------+---------------+---------------+
|pickup_location|  avg_transaction|max_transaction|min_transaction|
+---------------+-----------------+---------------+---------------+
|           1959|841.8840579710145|         3600.0|           60.0|
|           8928|658.6363636363636|         3200.0|           30.0|
|           1507|741.8181818181819|         3250.0|           40.0|
|           8932|829.0769230769231|         3500.0|           40.0|
|           9182|            852.0|         3450.0|           20.0|
+---------------+-----------------+---------------+---------------+
only showing top 5 rows



In [17]:
# Unique Vehicles Used at Each Location
unique_vehicles_per_location_df = transactions.groupBy("pickup_location").agg(F.countDistinct("vehicle_id").alias("unique_vehicles"))
unique_vehicles_per_location_df.show(5)

+---------------+---------------+
|pickup_location|unique_vehicles|
+---------------+---------------+
|           1959|             69|
|           8928|             66|
|           9182|             75|
|           8932|             65|
|           1507|             77|
+---------------+---------------+
only showing top 5 rows



In [23]:
transactions_with_vehicle_type_df = transactions.alias("t").join(
    vehicles.alias("v"), F.col("t.vehicle_id") == F.col("v.vehicle_id"), "left"
).select(
    F.col("t.rental_id").alias("rental_id"),
    F.col("t.user_id").alias("user_id"),
    F.col("t.vehicle_id").alias("vehicle_id"),
    F.col("t.rental_start_time").alias("rental_start_time"),
    F.col("t.rental_end_time").alias("rental_end_time"),
    F.col("t.pickup_location").alias("pickup_location"),
    F.col("t.dropoff_location").alias("dropoff_location"),
    F.col("t.total_amount").alias("total_amount"),
    F.col("v.vehicle_type").alias("vehicle_type")
)
transactions_with_vehicle_type_df.show(5)

+----------+----------+----------+-------------------+-------------------+---------------+----------------+------------+------------+
| rental_id|   user_id|vehicle_id|  rental_start_time|    rental_end_time|pickup_location|dropoff_location|total_amount|vehicle_type|
+----------+----------+----------+-------------------+-------------------+---------------+----------------+------------+------------+
|b139d8e1b2|320be8068b|0d52304987|2024-02-28 08:05:00|2024-03-01 05:05:00|           1497|            6785|       450.0|       basic|
|7afd60f6d3|320be8068b|975d72985c|2024-01-07 20:16:00|2024-01-09 21:16:00|           5345|            2608|      2450.0|    high_end|
|733a9361bc|8f31b734a6|0d9f0f0fb9|2024-01-07 09:36:00|2024-01-07 17:36:00|           2546|            5442|        80.0|       basic|
|6e546b69dd|8f31b734a6|967fdab45e|2024-01-05 11:30:00|2024-01-07 04:30:00|           8147|            4380|      2050.0|    high_end|
|acc192b64a|8f31b734a6|32d58ea4b7|2024-03-06 18:19:00|2024-03-

In [29]:
# Rental Duration and Revenue by Location
# Calculate rental duration in hours and aggregate revenue and duration by location
rental_duration_revenue_by_location_df = transactions.withColumn(
    "rental_duration_hours", 
    (F.col("rental_end_time").cast("long") - F.col("rental_start_time").cast("long")) / 3600
).groupBy("pickup_location").agg(
    F.sum("total_amount").alias("total_revenue_by_location"),
    F.sum("rental_duration_hours").alias("total_rental_duration_by_location")
)
rental_duration_revenue_by_location_df.show(5)

+---------------+-------------------------+---------------------------------+
|pickup_location|total_revenue_by_location|total_rental_duration_by_location|
+---------------+-------------------------+---------------------------------+
|           1959|                  58090.0|                           2770.0|
|           8928|                  43470.0|                           2230.0|
|           1507|                  57120.0|                           2897.0|
|           8932|                  53890.0|                           2318.0|
|           9182|                  63900.0|                           2626.0|
+---------------+-------------------------+---------------------------------+
only showing top 5 rows



In [31]:
# Join all KPIs on location (and vehicle_type where applicable)

final_kpi_df = revenue_per_location_df \
        .join(transactions_per_location_df, "pickup_location", "left") \
        .join(transaction_amounts_df, "pickup_location", "left") \
        .join(unique_vehicles_per_location_df, "pickup_location", "left") \
        .join(rental_duration_revenue_by_location_df, "pickup_location", "left")

final_kpi_df.show(5)

+---------------+-------------+------------------+-----------------+---------------+---------------+---------------+-------------------------+---------------------------------+
|pickup_location|total_revenue|total_transactions|  avg_transaction|max_transaction|min_transaction|unique_vehicles|total_revenue_by_location|total_rental_duration_by_location|
+---------------+-------------+------------------+-----------------+---------------+---------------+---------------+-------------------------+---------------------------------+
|           1959|      58090.0|                69|841.8840579710145|         3600.0|           60.0|             69|                  58090.0|                           2770.0|
|           8928|      43470.0|                66|658.6363636363636|         3200.0|           30.0|             66|                  43470.0|                           2230.0|
|           1507|      57120.0|                77|741.8181818181819|         3250.0|           40.0|             77

### Spark Job 2